In [1]:
# =============================================================================
# BLOCK 1: SETUP, IMPORTS, AND DATA LOADING
# =============================================================================
import warnings
warnings.filterwarnings('ignore')
import time

# --- Library Imports ---
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import optuna

print("Libraries imported successfully.")


# --- Helper Function for Winkler Score ---
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    width = upper - lower
    penalty_lower = np.where(y_true < lower, (2 / alpha) * (lower - y_true), 0)
    penalty_upper = np.where(y_true > upper, (2 / alpha) * (y_true - upper), 0)
    score = width + penalty_lower + penalty_upper
    if return_coverage:
        coverage = np.mean((y_true >= lower) & (y_true <= upper))
        return np.mean(score), coverage
    return np.mean(score)

# --- Global Constants ---
N_SPLITS = 5
RANDOM_STATE = 42
DATA_PATH = './'
N_OPTUNA_TRIALS = 75 # A strong number for a comprehensive search
COMPETITION_ALPHA = 0.1

# --- Load Raw Data ---
try:
    # We drop the low-variance columns they identified right away
    drop_cols=['id', 'golf', 'view_rainier', 'view_skyline', 'view_lakesamm', 'view_otherwater', 'view_other']
    df_train = pd.read_csv(DATA_PATH + 'dataset.csv').drop(columns=drop_cols)
    df_test = pd.read_csv(DATA_PATH + 'test.csv').drop(columns=drop_cols)
    print("Raw data loaded successfully.")
except FileNotFoundError:
    print("ERROR: Could not find 'dataset.csv' or 'test.csv'.")
    exit()

# --- Prepare Target Variable ---
y_true = df_train['sale_price'].copy()
# The mean-error model works best when predicting the raw price directly
# So, we will NOT log-transform the target this time.
# df_train.drop('sale_price', axis=1, inplace=True) # We keep sale_price for FE

print("Setup complete.")

Libraries imported successfully.
Raw data loaded successfully.
Setup complete.


In [2]:
# =============================================================================
# BLOCK 2: SYNTHESIZED FEATURE ENGINEERING (CORRECTED)
# =============================================================================
print("--- Starting Block 2: Synthesized Feature Engineering ---")

def create_synthesized_features(df_train, df_test):
    # Combine for consistent processing and reset the index
    df_train['is_train'] = 1
    df_test['is_train'] = 0
    # Store the original id for later, as reset_index will remove it
    train_ids = df_train.index
    test_ids = df_test.index
    all_data = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)

    # --- A) Brute-Force Numerical Interactions ---
    print("Creating brute-force numerical interaction features...")
    NUMS = ['area', 'land_val', 'imp_val', 'sqft_lot', 'sqft', 'sqft_1', 'grade', 'year_built']
    for i in range(len(NUMS)):
        for j in range(i + 1, len(NUMS)):
            all_data[f'{NUMS[i]}_x_{NUMS[j]}'] = all_data[NUMS[i]] * all_data[NUMS[j]]

    # --- B) Date Features ---
    all_data['sale_date'] = pd.to_datetime(all_data['sale_date'])
    all_data['year'] = all_data['sale_date'].dt.year
    all_data['month'] = all_data['sale_date'].dt.month
    all_data['year_diff'] = all_data['year'] - all_data['year_built']

    # --- C) TF-IDF Text Features ---
    print("Creating TF-IDF features for text columns...")
    text_cols = ['subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
    all_data[text_cols] = all_data[text_cols].fillna('missing').astype(str)

    for col in text_cols:
        tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 5), max_features=128, binary=True)
        tfidf_matrix = tfidf.fit_transform(all_data[col])
        svd = TruncatedSVD(n_components=8, random_state=RANDOM_STATE)
        tfidf_svd = svd.fit_transform(tfidf_matrix)
        tfidf_df = pd.DataFrame(tfidf_svd, columns=[f'{col}_tfidf_svd_{i}' for i in range(8)])
        # This concat will now work because both have a simple 0-based index
        all_data = pd.concat([all_data, tfidf_df], axis=1)

    # --- D) Log transform some of the new interaction features ---
    for c in ['land_val_x_imp_val', 'land_val_x_sqft', 'imp_val_x_sqft']:
        if c in all_data.columns:
            # Add a small constant to avoid log(0)
            all_data[c] = np.log1p(all_data[c].fillna(0))
            
    # --- E) Final Cleanup ---
    print("Finalizing feature set...")
    cols_to_drop = ['sale_date', 'subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
    all_data = all_data.drop(columns=cols_to_drop)
    all_data.fillna(0, inplace=True)

    # Separate final datasets
    X = all_data[all_data['is_train'] == 1].drop(columns=['is_train', 'sale_price'])
    X_test = all_data[all_data['is_train'] == 0].drop(columns=['is_train', 'sale_price'])
    
    # Restore the original 'id' as the index
    X.index = train_ids
    X_test.index = test_ids
    
    X_test = X_test[X.columns]
    
    return X, X_test

# We need to re-run this from the original dataframes
X, X_test = create_synthesized_features(df_train, df_test)

print(f"\nSynthesized FE complete. Total features: {X.shape[1]}")
gc.collect()

--- Starting Block 2: Synthesized Feature Engineering ---
Creating brute-force numerical interaction features...
Creating TF-IDF features for text columns...
Finalizing feature set...

Synthesized FE complete. Total features: 111


10

In [3]:
# =============================================================================
# BLOCK 3: TWO-STAGE TUNING, TRAINING, AND SUBMISSION
# =============================================================================

print("\n--- Starting Block 3: Two-Stage Modeling Pipeline ---")

# --- STAGE 1, PART 1: Tuning Mean Prediction Model ---
print("\n# STAGE 1, PART 1: Tuning Mean Prediction Model...")
def objective_mean(trial):
    train_x, val_x, train_y, val_y = train_test_split(X, y_true, test_size=0.2, random_state=RANDOM_STATE)
    params = {
            'objective': 'reg:squarederror', 
            'eval_metric': 'rmse', 
            'tree_method':'hist',
            'eta': trial.suggest_float('eta', 0.036, 0.042),
            'max_depth': trial.suggest_int('max_depth', 7, 8),
            'subsample': trial.suggest_float('subsample', 0.82, 0.85),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.84, 0.88),
            'lambda': trial.suggest_float('lambda', 3.1, 4.5),
            'alpha': trial.suggest_float('alpha', 1e-5, 5e-4, log=True),
            'min_child_weight': trial.suggest_int('min_child_weight', 2, 3)
        }
    
    model = xgb.XGBRegressor(**params, n_estimators=2500, random_state=RANDOM_STATE, n_jobs=-1, early_stopping_rounds=100)
    model.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=False)
    preds = model.predict(val_x)
    return np.sqrt(mean_squared_error(val_y, preds))

study_mean = optuna.create_study(direction='minimize')
study_mean.optimize(objective_mean, n_trials=N_OPTUNA_TRIALS)
best_params_mean = study_mean.best_params
print(f"# Mean Model Tuning Complete. Best Validation RMSE: ${study_mean.best_value:,.2f}")

# --- STAGE 1, PART 2: K-Fold Training of Mean Model ---
print("\n# STAGE 1, PART 2: K-Fold Training of Mean Model...")
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
oof_mean_preds = np.zeros(len(X))
test_mean_preds = np.zeros(len(X_test))
grade_for_stratify = pd.read_csv(DATA_PATH + 'dataset.csv')['grade']
for fold, (train_idx, val_idx) in enumerate(skf.split(X, grade_for_stratify)):
    print(f"  Mean Model - Fold {fold+1}/{N_SPLITS}...")
    model = xgb.XGBRegressor(**best_params_mean, n_estimators=2500, objective='reg:squarederror', eval_metric='rmse', tree_method='hist', random_state=RANDOM_STATE, n_jobs=-1, early_stopping_rounds=100)
    model.fit(X.iloc[train_idx], y_true.iloc[train_idx], eval_set=[(X.iloc[val_idx], y_true.iloc[val_idx])], verbose=False)
    oof_mean_preds[val_idx] = model.predict(X.iloc[val_idx])
    test_mean_preds += model.predict(X_test) / N_SPLITS
    
# --- NEW: CALCULATE AND PRINT FINAL OOF RMSE ---
final_mean_rmse = np.sqrt(mean_squared_error(y_true, oof_mean_preds))
print(f"\n# Mean model K-Fold training complete.")
print(f"# Final OOF RMSE for Mean Model: ${final_mean_rmse:,.2f}")
print("-" * 50)

[I 2025-07-08 15:31:29,839] A new study created in memory with name: no-name-f749b88a-5870-4d5d-be06-3c050dabe3b4



--- Starting Block 3: Two-Stage Modeling Pipeline ---

# STAGE 1, PART 1: Tuning Mean Prediction Model...


[I 2025-07-08 15:32:18,259] Trial 0 finished with value: 99182.80108970507 and parameters: {'eta': 0.04130190278041712, 'max_depth': 8, 'subsample': 0.822537243651096, 'colsample_bytree': 0.8792855060373929, 'lambda': 4.093488781549963, 'alpha': 3.2492961864610686e-05, 'min_child_weight': 3}. Best is trial 0 with value: 99182.80108970507.
[I 2025-07-08 15:32:59,901] Trial 1 finished with value: 98779.02935340072 and parameters: {'eta': 0.03642760312245598, 'max_depth': 8, 'subsample': 0.8255260631925442, 'colsample_bytree': 0.875883364385171, 'lambda': 4.272223040858798, 'alpha': 1.9717048288526794e-05, 'min_child_weight': 3}. Best is trial 1 with value: 98779.02935340072.
[I 2025-07-08 15:33:37,180] Trial 2 finished with value: 99079.95833668885 and parameters: {'eta': 0.03815715328767762, 'max_depth': 8, 'subsample': 0.8365532897030313, 'colsample_bytree': 0.8792208379038297, 'lambda': 3.17238430423023, 'alpha': 6.283357201642059e-05, 'min_child_weight': 3}. Best is trial 1 with valu

# Mean Model Tuning Complete. Best Validation RMSE: $98,514.29

# STAGE 1, PART 2: K-Fold Training of Mean Model...
  Mean Model - Fold 1/5...
  Mean Model - Fold 2/5...
  Mean Model - Fold 3/5...
  Mean Model - Fold 4/5...
  Mean Model - Fold 5/5...

# Mean model K-Fold training complete.
# Final OOF RMSE for Mean Model: $98,985.22
--------------------------------------------------


In [4]:
# --- STAGE 2, PART 1: NEW - Tuning the Error Model ---
print("\n# STAGE 2, PART 1: Tuning Error Prediction Model...")
print("# EXPLANATION: We are now finding the best settings for the second model, which predicts the error.")
error_target = np.abs(y_true - oof_mean_preds)
X_for_error = X.copy()
X_for_error['mean_pred_oof'] = oof_mean_preds

def objective_error(trial):
    train_x, val_x, train_y, val_y = train_test_split(X_for_error, error_target, test_size=0.2, random_state=RANDOM_STATE)
    # The error model might benefit from different, simpler parameters
    params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'tree_method': 'hist',
            'eta': trial.suggest_float('eta', 0.014, 0.019),
            'max_depth': trial.suggest_int('max_depth', 8, 10),
            'subsample': trial.suggest_float('subsample', 0.9, 0.95),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.725, 0.745),
            'lambda': trial.suggest_float('lambda', 0.5, 0.7, log=True),
            'alpha': trial.suggest_float('alpha', 0.55, 0.63),
        }


    
    model = xgb.XGBRegressor(**params, n_estimators=2500, random_state=RANDOM_STATE, n_jobs=-1, early_stopping_rounds=100)
    model.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=False)
    preds = model.predict(val_x)
    return np.sqrt(mean_squared_error(val_y, preds))

study_error = optuna.create_study(direction='minimize')
study_error.optimize(objective_error, n_trials=N_OPTUNA_TRIALS)
best_params_error = study_error.best_params
print(f"# Error Model Tuning Complete. Best Validation RMSE: ${study_error.best_value:,.2f}")

# --- STAGE 2, PART 2: K-Fold Training of Error Model with Best Params ---
print("\n# STAGE 2, PART 2: K-Fold Training of Error Model...")
X_test_for_error = X_test.copy()
X_test_for_error['mean_pred_oof'] = test_mean_preds
oof_error_preds = np.zeros(len(X))
test_error_preds = np.zeros(len(X_test))
for fold, (train_idx, val_idx) in enumerate(skf.split(X_for_error, grade_for_stratify)):
    print(f"  Error Model - Fold {fold+1}/{N_SPLITS}...")
    # Use the NEW best parameters for the error model
    model = xgb.XGBRegressor(**best_params_error, n_estimators=2500, objective='reg:squarederror', eval_metric='rmse', tree_method='hist', random_state=RANDOM_STATE, n_jobs=-1, early_stopping_rounds=100)
    model.fit(X_for_error.iloc[train_idx], error_target.iloc[train_idx], eval_set=[(X_for_error.iloc[val_idx], error_target.iloc[val_idx])], verbose=False)
    oof_error_preds[val_idx] = model.predict(X_for_error.iloc[val_idx])
    test_error_preds += model.predict(X_test_for_error) / N_SPLITS

# --- NEW: CALCULATE AND PRINT FINAL OOF RMSE ---
final_error_rmse = np.sqrt(mean_squared_error(error_target, oof_error_preds))
print(f"\n# Error model K-Fold training complete.")
print(f"# Final OOF RMSE for Error Model: ${final_error_rmse:,.2f}")
print("-" * 50)


[I 2025-07-08 16:23:59,851] A new study created in memory with name: no-name-e73a9b73-78cf-44f5-97a0-4c70ffd16362



# STAGE 2, PART 1: Tuning Error Prediction Model...
# EXPLANATION: We are now finding the best settings for the second model, which predicts the error.


[I 2025-07-08 16:24:13,538] Trial 0 finished with value: 63327.49021311475 and parameters: {'eta': 0.018798886721168768, 'max_depth': 9, 'subsample': 0.9446176861998369, 'colsample_bytree': 0.7290779264704449, 'lambda': 0.5465441996286214, 'alpha': 0.5982650785082044}. Best is trial 0 with value: 63327.49021311475.
[I 2025-07-08 16:24:28,706] Trial 1 finished with value: 63485.4903438292 and parameters: {'eta': 0.016163975041349865, 'max_depth': 10, 'subsample': 0.9368270416357979, 'colsample_bytree': 0.7292634594952883, 'lambda': 0.6246117866330293, 'alpha': 0.6190856769596473}. Best is trial 0 with value: 63327.49021311475.
[I 2025-07-08 16:24:51,747] Trial 2 finished with value: 63312.03203755258 and parameters: {'eta': 0.014298199772128093, 'max_depth': 9, 'subsample': 0.9250735183615486, 'colsample_bytree': 0.7384195983232821, 'lambda': 0.6249517679709083, 'alpha': 0.5536560194801364}. Best is trial 2 with value: 63312.03203755258.
[I 2025-07-08 16:25:11,744] Trial 3 finished with

# Error Model Tuning Complete. Best Validation RMSE: $63,236.51

# STAGE 2, PART 2: K-Fold Training of Error Model...
  Error Model - Fold 1/5...
  Error Model - Fold 2/5...
  Error Model - Fold 3/5...
  Error Model - Fold 4/5...
  Error Model - Fold 5/5...

# Error model K-Fold training complete.
# Final OOF RMSE for Error Model: $62,811.13
--------------------------------------------------


In [5]:
# =============================================================================
# FINAL ASYMMETRIC CALIBRATION AND SUBMISSION (ULTIMATE ROBUST VERSION)
# =============================================================================

print("\n--- Final Asymmetric Calibration ---")

# --- Safely reload y_true to ensure it's available ---
y_true = pd.read_csv('./dataset.csv')['sale_price']

# --- Your existing correct code ---
oof_error_final = np.clip(oof_error_preds, 0, None) 
best_a, best_b, best_metric = 2.0, 2.0, float('inf')

for a in np.arange(1.90, 2.31, 0.01):
    for b in np.arange(2.10, 2.51, 0.01):
        low = oof_mean_preds - oof_error_final * a
        high = oof_mean_preds + oof_error_final * b
        # We need the winkler_score function defined here or in a previous cell
        metric, coverage = winkler_score(y_true, low, high, alpha=COMPETITION_ALPHA, return_coverage=True)
        if metric < best_metric:
            best_metric = metric
            best_a, best_b = a, b
            
print(f"\nGrid search complete. Final OOF Score: {best_metric:,.2f}. Best multipliers: a={best_a:.2f}, b={best_b:.2f}")


# --- Create Final Submission ---
print("\nCreating final submission file...")
test_error_final = np.clip(test_error_preds, 0, None)
final_lower = test_mean_preds - test_error_final * best_a
final_upper = test_mean_preds + test_error_final * best_b
final_upper = np.maximum(final_lower, final_upper)

# Your excellent, robust fix for the IDs
test_ids = pd.read_csv('./test.csv', usecols=['id'])['id']
submission_df = pd.DataFrame({
    'id': test_ids, 
    'pi_lower': final_lower, 
    'pi_upper': final_upper
})

submission_df.to_csv('submission_final_v6.csv', index=False)
print("\n'submission_final_v6.csv' created successfully!")
display(submission_df.head())


--- Final Asymmetric Calibration ---

Grid search complete. Final OOF Score: 302,580.65. Best multipliers: a=1.96, b=2.17

Creating final submission file...

'submission_final_v6.csv' created successfully!


,id,pi_lower,pi_upper
0,200000,835040.845977,1.059650e+06
1,200001,601868.057109,8.297536e+05
2,200002,450406.289844,6.572032e+05
3,200003,281483.373613,4.069948e+05
4,200004,295944.036016,8.227563e+05
